#Nariman Zendehrooh Kermani

##Imports

In [1]:
import numpy as np
import pandas as pd
import psycopg2
from datetime import datetime

##Data Extraction

In [2]:
ds = pd.read_csv('https://data-engineer-technical-challenge.s3.ap-southeast-2.amazonaws.com/pace-data.txt')

In [3]:
ds.shape

(174058, 20)

In [4]:
ds.columns

Index(['MovementDateTime', 'Destination', 'DestinationTidied', 'Speed',
       'AdditionalInfo', 'CallSign', 'Heading', 'MMSI', 'MovementID',
       'ShipName', 'ShipType', 'Beam', 'Draught', 'Length', 'ETA',
       'MoveStatus', 'ladenStatus', 'LRIMOShipNo', 'Latitude', 'Longitude'],
      dtype='object')

In [5]:
ds.head(10)

,MovementDateTime,Destination,DestinationTidied,Speed,AdditionalInfo,CallSign,Heading,MMSI,MovementID,ShipName,ShipType,Beam,Draught,Length,ETA,MoveStatus,ladenStatus,LRIMOShipNo,Latitude,Longitude
0,2019-07-20 00:00:00,PATIMBAN,NaN,9.8,NaN,YDLR,90.0,525019348,4780574056319002,MV SENANG,Cargo,16,6.8,117,2019-07-20 16:00:00.000,Under way using engine,Ballast,7805746,-5.732933,107.444069
1,2019-07-20 00:00:00,LA PAZ,La Paz,0.0,NaN,XCAR8,163.0,345060069,4681115056319002,SAKIS P,Cargo,13,5.1,82,2019-07-18 18:30:00.000,Moored,Ballast,6811152,24.264002,-110.331352
2,2019-07-20 00:00:00,GTPRQ,Puerto Quetzal,13.3,NaN,V7A2042,118.0,538008172,4984813056319002,YASA JUPITER,Cargo,32,10.7,199,2019-07-21 10:00:00.000,Under way using engine,Ballast,9848132,15.103770,-97.482719
3,2019-07-20 00:00:00,RU LED,St Petersburg Ugolnaya,0.0,NaN,D5SQ4,321.0,636019034,4977443056319002,MEDI ADRIATICO,Cargo,32,6.4,200,2019-07-18 03:00:00.000,Anchored,Ballast,9774434,60.105572,29.347267
4,2019-07-20 00:00:00,KAMSAR,Port Kamsar,13.6,NaN,3FBS5,216.0,372798000,4978315056319002,SAKIZAYA RESPECT,Cargo,32,6.8,229,2019-07-24 15:00:00.000,Under way using engine,Ballast,9783150,29.713890,-12.162312
5,2019-07-20 00:00:00,CNDLC,Dalian,12.5,NaN,D5SM8,300.0,636019011,4978396056319002,STABILITY,Cargo,32,12.9,229,2019-08-11 22:00:00.000,Under way using engine,Ballast,9783966,22.984583,-113.032501
6,2019-07-20 00:00:00,ID CIG __AU PWL,NaN,12.8,NaN,9V5142,175.0,563029100,4980121056319002,CSK UNITED,Cargo,50,9.2,299,2019-07-23 21:00:00.000,Under way using engine,Ballast,9801213,-6.572037,105.007835
7,2019-07-20 00:00:00,SAO FRANCISCO DO SUL,Sao Francisco do Sul,9.5,NaN,9V6080,241.0,563079400,4983646056319002,PREDATOR,Cargo,32,7.1,229,2019-08-14 08:00:00.000,Under way using engine,Ballast,9836464,0.335243,86.707047
8,2019-07-20 00:00:00,CALLAO_PERU,Callao,11.9,NaN,V7JP6,130.0,538006084,4973511056319002,SWANSEA,Cargo,32,13.3,199,2019-07-24 23:55:00.000,Under way using engine,Ballast,9735115,5.829837,-93.069611
9,2019-07-20 00:00:00,LIVERPOOL_UK,Liverpool (United Kingdom),11.9,NaN,V7ZT5,345.0,538007228,4973580056319002,GH RICH WALL,Cargo,32,10.8,199,2019-07-23 06:00:00.000,Under way using engine,Ballast,9735804,37.653912,-10.163797


In [6]:
ds.isnull().sum().sort_values(ascending=False)

AdditionalInfo       172830
DestinationTidied     40703
ShipType               2567
Destination             695
MoveStatus              462
CallSign                 49
ShipName                  1
Length                    0
Latitude                  0
LRIMOShipNo               0
ladenStatus               0
ETA                       0
MovementDateTime          0
Draught                   0
Beam                      0
MovementID                0
MMSI                      0
Heading                   0
Speed                     0
Longitude                 0
dtype: int64

##MovementDateTime Normalization

In [7]:
ds['MovementDateTime'] = pd.to_datetime(ds['MovementDateTime'])
ds['MovementDateTime'] = ds['MovementDateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

##CallSign Processing

In [8]:
ds.loc[ds["MoveStatus"] == "Under way using engine"].loc[ds["Speed"] == 0.0]

,MovementDateTime,Destination,DestinationTidied,Speed,AdditionalInfo,CallSign,Heading,MMSI,MovementID,ShipName,ShipType,Beam,Draught,Length,ETA,MoveStatus,ladenStatus,LRIMOShipNo,Latitude,Longitude
105,2019-07-20T00:03:00,BENGKULU_IND,NaN,0.0,NaN,DSQZ4,162.0,441763000,4948924056319002,CH DORIS,Cargo,28,6.2,177,2019-07-03 13:00:00.000,Under way using engine,Ballast,9489247,-4.838583,103.397552
124,2019-07-20T00:04:00,TG.PRIOK,Tanjung Priok,0.0,NaN,JZIG,75.0,525019643,4739702056319002,MV GIAT,Cargo,16,5.0,118,2019-07-20 06:00:00.000,Under way using engine,Ballast,7397024,-6.039500,106.900169
131,2019-07-20T00:04:00,SGSIN.PEBGC,Singapore,0.0,NaN,DSRK3,89.0,441889000,4914980056319002,SINOMERCHANT,Cargo,58,6.1,332,2019-07-18 07:00:00.000,Under way using engine,Ballast,9149809,1.251810,103.609467
423,2019-07-20T00:14:00,HCM VN,Saigon East,0.0,NaN,3WCV9,0.0,574001010,4959979056318551,LONG TAN 99,NaN,15,7.2,91,2019-12-08 15:00:00.000,Under way using engine,Ballast,9599793,10.586783,107.028656
531,2019-07-20T00:18:00,IMBITUBA,Imbituba,0.0,NaN,V7NM2,331.0,538006478,4975409056319002,INTERLINK AUDACITY,Cargo,30,9.6,180,2019-07-17 02:45:00.000,Under way using engine,Ballast,9754094,-28.226669,-48.651669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173862,2019-07-20T23:59:00,SGSIN.PEBGC,Singapore,0.0,NaN,DSRK3,89.0,441889000,4914980056332801,SINOMERCHANT,Cargo,58,6.1,332,2019-07-18 07:00:00.000,Under way using engine,Ballast,9149809,1.251805,103.609451
173867,2019-07-20T23:59:00,CLARKSON,Clarkson,0.0,NaN,CFN4934,315.0,316011905,4736640056332801,ROBERT S. PIERSON,Cargo,21,7.9,192,2019-07-20 19:00:00.000,Under way using engine,Ballast,7366403,43.483795,-79.609123
173875,2019-07-20T23:59:00,CILACAP,Tanjung Intan,0.0,NaN,PLVX,20.0,525018002,4751854056332801,MV IBRAHIM ZAHIER,Cargo,20,6.1,116,2019-07-18 21:00:00.000,Under way using engine,Ballast,7518549,-7.734228,108.991364
174007,2019-07-20T23:59:00,KILIFI,NaN,0.0,NaN,HOFY,107.0,374009000,4977412056333402,SI HANG YUN SHA 2,Cargo,19,3.2,87,2019-11-09 07:00:00.000,Under way using engine,Ballast,9774123,-3.637833,39.844856


In [9]:
def speed_processing(ds):
    m = ds['Speed'].mean().round(1)
    
    for index, row in ds.iterrows():
        if row["MoveStatus"] == "Under way using engine" and row["Speed"] == 0.0:
            ds.at[index, "Speed"] = m
    
    return ds

ds.groupby('CallSign', group_keys=False).apply(speed_processing)

,MovementDateTime,Destination,DestinationTidied,Speed,AdditionalInfo,CallSign,Heading,MMSI,MovementID,ShipName,ShipType,Beam,Draught,Length,ETA,MoveStatus,ladenStatus,LRIMOShipNo,Latitude,Longitude
0,2019-07-20T00:00:00,PATIMBAN,NaN,9.8,NaN,YDLR,90.000000,525019348,4780574056319002,MV SENANG,Cargo,16,6.800000,117,2019-07-20 16:00:00.000,Under way using engine,Ballast,7805746,-5.732933,107.444069
1,2019-07-20T00:00:00,LA PAZ,La Paz,0.0,NaN,XCAR8,163.000000,345060069,4681115056319002,SAKIS P,Cargo,13,5.100000,82,2019-07-18 18:30:00.000,Moored,Ballast,6811152,24.264002,-110.331352
2,2019-07-20T00:00:00,GTPRQ,Puerto Quetzal,13.3,NaN,V7A2042,118.000000,538008172,4984813056319002,YASA JUPITER,Cargo,32,10.700000,199,2019-07-21 10:00:00.000,Under way using engine,Ballast,9848132,15.103770,-97.482719
3,2019-07-20T00:00:00,RU LED,St Petersburg Ugolnaya,0.0,NaN,D5SQ4,321.000000,636019034,4977443056319002,MEDI ADRIATICO,Cargo,32,6.400000,200,2019-07-18 03:00:00.000,Anchored,Ballast,9774434,60.105572,29.347267
4,2019-07-20T00:00:00,KAMSAR,Port Kamsar,13.6,NaN,3FBS5,216.000000,372798000,4978315056319002,SAKIZAYA RESPECT,Cargo,32,6.800000,229,2019-07-24 15:00:00.000,Under way using engine,Ballast,9783150,29.713890,-12.162312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174053,2019-07-20T23:59:00,VISAKHAPATNAM,Visakhapatnam,9.6,NaN,VRQO5,53.000000,477333200,4975242056332802,DARYA SATI,Cargo,32,11.900000,200,2019-07-30 22:30:00.000,Under way using engine,Ballast,9752424,-16.734249,60.068554
174054,2019-07-20T23:59:00,CN TAPAN CHINA,NaN,9.0,NaN,D5KU7,350.000000,636017413,4974789056332801,TRUE PATRIOT,Cargo,45,18.200001,292,2019-07-18 14:00:00.000,Under way using engine,Ballast,9747895,38.954613,119.223076
174055,2019-07-20T23:59:00,KR PUS,Busan,12.9,NaN,7JWB,206.000000,431287000,4974802056332791,TOHOKUMARU,Cargo,38,8.400000,230,2019-07-22 10:00:00.000,Under way using engine,Ballast,9748021,37.412926,136.370850
174056,2019-07-20T23:59:00,NORFOLK,Norfolk,0.0,NaN,V7OI7,116.699997,538006906,4974991056332791,CHLOE,Cargo,33,8.200000,229,2019-07-18 01:00:00.000,Anchored,Ballast,9749910,37.219616,-76.125488


##BeamRatio Creation

In [10]:
ds['BeamRatio'] = ds.apply(lambda x: x['Beam']/x['Length'] if x['Length'] != 0 else 0, axis=1)

##Cleaned Data Export

In [11]:
ds.to_csv('/tmp/enriched.csv') 

##Database Creation

In [12]:
conn = psycopg2.connect(database="challenge", user='rio', password='riotinto', host='localhost', port='5433')
conn.autocommit = True
cursor = conn.cursor()

In [13]:
createsql = '''CREATE TABLE PACE(Index int NOT NULL, MovementDateTime char(19) NOT NULL, Destination varchar(30),\
DestinationTidied varchar(100), Speed float, AdditionalInfo varchar(100), CallSign varchar(20), Heading float,\
MMSI int, MovementID varchar(30), ShipName varchar(30), ShipType varchar(30), Beam int, Draught float, Length int,\
ETA varchar(30), MoveStatus varchar(30), ladenStatus varchar(30), LRIMOShipNo int, Latitude float, Longitude float, BeamRatio float);'''

cursor.execute(createsql)

In [14]:
csvcopysql = '''COPY pace(Index, MovementDateTime, Destination,\
DestinationTidied, Speed, AdditionalInfo, CallSign, Heading,\
MMSI, MovementID, ShipName, ShipType, Beam, Draught, Length,\
ETA, MoveStatus, ladenStatus, LRIMOShipNo, Latitude, Longitude, BeamRatio)
FROM '/tmp/enriched.csv'
DELIMITER ','
CSV HEADER;'''

cursor.execute(csvcopysql)

In [15]:
selectsql = ''' SELECT * FROM pace LIMIT 20;'''

cursor.execute(selectsql)
row = cursor.fetchone()
while row is not None:
  print(row)
  row = cursor.fetchone()

(0, '2019-07-20T00:00:00', 'PATIMBAN', None, 9.800000190734863, None, 'YDLR', 90.0, 525019348, '4780574056319002', 'MV SENANG', 'Cargo', 16, 6.800000190734863, 117, '2019-07-20 16:00:00.000', 'Under way using engine', 'Ballast', 7805746, -5.732933044433594, 107.4440689086914, 0.13675213675213677)
(1, '2019-07-20T00:00:00', 'LA PAZ', 'La Paz', 0.0, None, 'XCAR8', 163.0, 345060069, '4681115056319002', 'SAKIS P', 'Cargo', 13, 5.099999904632568, 82, '2019-07-18 18:30:00.000', 'Moored', 'Ballast', 6811152, 24.26400184631348, -110.33135223388672, 0.15853658536585366)
(2, '2019-07-20T00:00:00', 'GTPRQ', 'Puerto Quetzal', 13.300000190734863, None, 'V7A2042', 118.0, 538008172, '4984813056319002', 'YASA JUPITER', 'Cargo', 32, 10.699999809265137, 199, '2019-07-21 10:00:00.000', 'Under way using engine', 'Ballast', 9848132, 15.10377025604248, -97.48271942138672, 0.16080402010050251)
(3, '2019-07-20T00:00:00', 'RU LED', 'St Petersburg Ugolnaya', 0.0, None, 'D5SQ4', 321.0, 636019034, '49774430563190

##Create S3 Bucket and Upload The CSV File

In [16]:
!aws --version

aws-cli/2.5.8 Python/3.9.11 Darwin/22.1.0 exe/x86_64 prompt/off


In [17]:
!aws s3 mb s3://riotinto-challenge --region ca-central-1
!aws s3 cp /tmp/enriched.csv s3://riotinto-challenge

make_bucket: riotinto-challenge
upload: ../../tmp/enriched.csv to s3://riotinto-challenge/enriched.csv


##Clean up

In [18]:
droptable = '''DROP TABLE pace'''
cursor.execute(droptable)

In [19]:
!aws s3 rm s3://riotinto-challenge/enriched.csv

delete: s3://riotinto-challenge/enriched.csv


In [20]:
!aws s3api delete-bucket --bucket "riotinto-challenge"